# 🚲 서울시 자전거 이용 데이터 분석
본 프로젝트에서는 2024년 12월 서울시 공공자전거(따릉이) 데이터와 날씨 데이터를 활용하여

**시간대/요일별 이용량**, **인기 대여소/반납소**, **계절별 변화**, **날씨와의 상관관계**를 분석합니다.

데이터 분석에 앞서 데이터는 서울시 열린데이터 광장에서 시간대별 자전거 데이터(12월)와 날씨 데이터를 사용하기 위한 준비 과정입니다.

## 1. 자전거 데이터 처리

In [25]:
# 데이터 불러오기
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# CSV 파일 경로 설정
file_path = '../datas/서울특별시 공공자전거 이용정보(시간대별)_202412.csv'

# CSV 파일 읽기 (cp949 인코딩 사용)
data = pd.read_csv(file_path, encoding='cp949')

# 데이터 확인
print(data.head())


         대여일자  대여시간  대여소번호                 대여소명 대여구분코드   성별 연령대코드  이용건수   
0  2024-12-01     0   1442  1442. 중랑구청 중화동 별관 앞    정기권  NaN  ~10대     1  \
1  2024-12-01     0   2728     2728.마곡나루역 3번 출구    정기권  NaN  ~10대     1   
2  2024-12-01     0   1023     1023. 한국종합기술사옥 앞    정기권  NaN   20대     1   
3  2024-12-01     0   1150       1150. 송정역 1번출구    정기권  NaN   20대     1   
4  2024-12-01     0   1260  1260. 방이동 한양3차아파트 옆    정기권  NaN   20대     1   

      운동량   탄소량  이동거리(M)  이용시간(분)  
0  105.32  0.66  2829.30       18  
1   22.45  0.25  1090.00        7  
2  148.31  0.87  3745.23       21  
3   24.01  0.19   808.44        6  
4   92.85  0.94  4042.55       35  


In [26]:
# 대여일자 + 대여시간 

data['대여일시'] = pd.to_datetime(data['대여일자'].astype(str) + ' ' + data['대여시간'].astype(str).str.zfill(2) + ':00')

# 확인
print(data.sample(5))

               대여일자  대여시간  대여소번호                 대여소명 대여구분코드   성별 연령대코드  이용건수   
1603510  2024-12-23    11    213             213. KT앞    정기권  NaN   20대     1  \
52023    2024-12-01    20   3569          3569.건대병원후문    일일권    M   20대     1   
606826   2024-12-09     7   1171  1171. 염창동 새마을금고 건너편    정기권    M  ~10대     1   
446842   2024-12-06    15   2907          2907. 월계중학교    정기권    F  ~10대     2   
1469478  2024-12-20    14   1957     1957. 구일고등학교 정문     정기권    M   30대     1   

           운동량   탄소량  이동거리(M)  이용시간(분)                대여일시  
1603510  25.91  0.22   962.07        8 2024-12-23 11:00:00  
52023    69.00  0.61  2640.00       14 2024-12-01 20:00:00  
606826   37.58  0.34  1460.00       14 2024-12-09 07:00:00  
446842   80.44  0.87  3738.50       24 2024-12-06 15:00:00  
1469478  79.28  0.71  3080.00       22 2024-12-20 14:00:00  


In [27]:
# 필요한 칼럼 선택
selected_columns = [
    '대여일시', '대여소명', '대여구분코드', '성별', '연령대코드', '이용건수', 
    '운동량', '탄소량', '이동거리(M)', '이용시간(분)'
]
data = data[selected_columns]

print(data.sample(5))

                       대여일시                     대여소명 대여구분코드   성별  연령대코드  이용건수   
455396  2024-12-06 16:00:00           806. 전자랜드 본관 앞    정기권    M    40대     1  \
1029135 2024-12-14 01:00:00         3778. 김포공항입구 교차로    정기권  NaN  70대이상     1   
1022604 2024-12-13 22:00:00  1503. 강북구청사거리(수유역 8번출구)    정기권  NaN    20대     1   
911417  2024-12-12 16:00:00         1754. 창동시장입구 사거리    정기권    M   ~10대     1   
834309  2024-12-11 18:00:00           5066. 아망떼마곡전시장    정기권    F    40대     1   

            운동량   탄소량  이동거리(M)  이용시간(분)  
455396   266.46  1.88  8106.94       87  
1029135   62.00  0.61  2609.46       15  
1022604  109.29  0.71  3066.61       18  
911417    32.25  0.24  1044.24        5  
834309    10.94  0.11   484.75        6  


In [28]:
# 요일과 시간대 칼럼 추가
data['요일'] = data['대여일시'].dt.day_name()  # 요일
data['시간대'] = data['대여일시'].dt.hour  # 시간대

print(data.sample(5))

                       대여일시                      대여소명 대여구분코드   성별 연령대코드  이용건수   
819364  2024-12-11 15:00:00               4864. 송파사거리    정기권    M   40대     1  \
930101  2024-12-12 19:00:00  2340. 삼호물산버스정류장(23370) 옆    정기권  NaN   30대     1   
2079258 2024-12-31 10:00:00            1708. 도봉보건소사거리    정기권    M   60대     1   
1829146 2024-12-26 22:00:00            1625. 상계역 3번출구    정기권  NaN   60대     1   
2087139 2024-12-31 13:00:00       2337. 대모산입구역 2번출구 앞    정기권    F  ~10대     1   

           운동량   탄소량  이동거리(M)  이용시간(분)         요일  시간대  
819364   53.03  0.50  2160.00       95  Wednesday   15  
930101   52.99  0.44  1884.85       24   Thursday   19  
2079258  28.49  0.26  1141.83        8    Tuesday   10  
1829146  28.24  0.25  1097.02       10   Thursday   22  
2087139   0.00  0.00     0.00       56    Tuesday   13  


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126133 entries, 0 to 2126132
Data columns (total 12 columns):
 #   Column   Dtype         
---  ------   -----         
 0   대여일시     datetime64[ns]
 1   대여소명     object        
 2   대여구분코드   object        
 3   성별       object        
 4   연령대코드    object        
 5   이용건수     int64         
 6   운동량      object        
 7   탄소량      object        
 8   이동거리(M)  float64       
 9   이용시간(분)  int64         
 10  요일       object        
 11  시간대      int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2), object(7)
memory usage: 186.5+ MB


In [30]:
print(data.isna().sum())

대여일시            0
대여소명            0
대여구분코드          0
성별         551865
연령대코드           0
이용건수            0
운동량             0
탄소량             0
이동거리(M)         0
이용시간(분)         0
요일              0
시간대             0
dtype: int64


In [31]:

data['성별'] = data['성별'].replace({'f': 'F', 'm': 'M', 'F': 'F', 'M': 'M'})

In [32]:
# 상위 폴더의 datas 폴더에 cleaned_bike.csv로 저장
output_file_path = '../datas/cleaned_bike.csv'
data.to_csv(output_file_path, index=False, encoding='cp949')

## 2. 기상 데이터 처리

In [33]:
# CSV 파일 경로 설정
file_path = '../datas/OBS_ASOS_TIM_20250510152404.csv'

# CSV 파일 읽기 (cp949 인코딩 사용)
df = pd.read_csv(file_path, encoding='cp949')

# 데이터 확인
print(df.head())

    지점 지점명                일시  기온(°C)  기온 QC플래그  강수량(mm)  강수량 QC플래그  풍속(m/s)   
0  108  서울  2024-12-01 00:00     2.8       NaN      NaN        9.0      1.5  \
1  108  서울  2024-12-01 01:00     2.5       NaN      NaN        NaN      1.9   
2  108  서울  2024-12-01 02:00     2.3       NaN      NaN        NaN      1.7   
3  108  서울  2024-12-01 03:00     1.9       NaN      NaN        NaN      2.5   
4  108  서울  2024-12-01 04:00     1.7       NaN      NaN        NaN      1.9   

   풍속 QC플래그  풍향(16방위)  ...  최저운고(100m )  시정(10m)  지면상태(지면상태코드)  현상번호(국내식)   
0       NaN        50  ...          NaN      512           NaN       19.0  \
1       NaN        20  ...          NaN      356           NaN       19.0   
2       NaN        20  ...          NaN      414           NaN       19.0   
3       NaN        20  ...          NaN      297           NaN       19.0   
4       NaN        50  ...          NaN      386           NaN       19.0   

   지면온도(°C)  지면온도 QC플래그  5cm 지중온도(°C)  10cm 지중온도(°C)  20cm 지중온

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   지점             744 non-null    int64  
 1   지점명            744 non-null    object 
 2   일시             744 non-null    object 
 3   기온(°C)         744 non-null    float64
 4   기온 QC플래그       0 non-null      float64
 5   강수량(mm)        23 non-null     float64
 6   강수량 QC플래그      148 non-null    float64
 7   풍속(m/s)        744 non-null    float64
 8   풍속 QC플래그       0 non-null      float64
 9   풍향(16방위)       744 non-null    int64  
 10  풍향 QC플래그       0 non-null      float64
 11  습도(%)          744 non-null    int64  
 12  습도 QC플래그       0 non-null      float64
 13  증기압(hPa)       744 non-null    float64
 14  이슬점온도(°C)      744 non-null    float64
 15  현지기압(hPa)      744 non-null    float64
 16  현지기압 QC플래그     0 non-null      float64
 17  해면기압(hPa)      744 non-null    float64
 18  해면기압 QC플래그

In [35]:
# '일시' 칼럼을 datetime 형식으로 변환
df['일시'] = pd.to_datetime(df['일시'])

# 자전거와 관련된 칼럼 선택 (예시: 기온, 강수량, 풍속 등)
# 필요한 칼럼 이름을 실제 데이터에 맞게 수정하세요.
bike_related_columns = [
    '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '지면온도(°C)', '적설(cm)'
]
df_bike = df[bike_related_columns]

In [36]:
df_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   일시        744 non-null    datetime64[ns]
 1   기온(°C)    744 non-null    float64       
 2   강수량(mm)   23 non-null     float64       
 3   풍속(m/s)   744 non-null    float64       
 4   습도(%)     744 non-null    int64         
 5   지면온도(°C)  744 non-null    float64       
 6   적설(cm)    84 non-null     float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 40.8 KB


In [37]:
df_bike.sample(5)

,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),지면온도(°C),적설(cm)
468,2024-12-20 12:00:00,2.7,NaN,2.2,50,0.1,NaN
573,2024-12-24 21:00:00,-0.3,NaN,1.1,50,-1.0,NaN
384,2024-12-17 00:00:00,-1.5,NaN,0.8,70,-2.3,NaN
576,2024-12-25 00:00:00,-0.3,NaN,1.8,52,-1.4,NaN
605,2024-12-26 05:00:00,3.3,NaN,3.0,76,-0.3,NaN


In [38]:
df_bike.fillna(0, inplace = True)

C:\Users\mj985\AppData\Local\Temp\ipykernel_2012\2445096506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bike.fillna(0, inplace = True)


In [39]:
df_bike.sample(5)

,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),지면온도(°C),적설(cm)
497,2024-12-21 17:00:00,-0.7,0.0,2.2,67,-0.2,1.4
671,2024-12-28 23:00:00,-3.1,0.0,0.6,56,-4.0,0.0
552,2024-12-24 00:00:00,-1.1,0.0,1.6,51,-2.4,0.0
729,2024-12-31 09:00:00,-1.5,0.0,2.0,49,-2.5,0.0
692,2024-12-29 20:00:00,3.0,0.0,1.7,63,-0.3,0.0


In [40]:
# DataFrame을 상위 폴더의 datas 폴더에 cleaned_weather.csv로 저장

output_file_path = '../datas/cleaned_weather.csv'
df_bike.to_csv(output_file_path, index=False, encoding='cp949')

In [ ]:
# 대여소 Top 10
top_start = bike['대여소명'].value_counts().head(10)
top_start.plot(kind='bar', title='Top 10 대여소', figsize=(8,4))
plt.show()

# 반납소 Top 10
top_end = bike['반납소명'].value_counts().head(10)
top_end.plot(kind='bar', color='orange', title='Top 10 반납소', figsize=(8,4))
plt.show()